In [1]:
                        #转换成ieee754标准
import numpy as np
from bitstring import BitArray

np.set_printoptions(suppress = True, threshold = np.inf, precision = 15)
import bitstring, random 
def ieee754H(flt):
    b = bitstring.BitArray(float=flt, length=32)
    return b
def ieee754B(flt):
    aa = bitstring.BitArray(float=flt, length=32)
    aa = str(aa)
    aa = aa[2:]
    aa = bin(int(aa, 16))
    aa = aa[2:]
    aa = aa.rjust(32,'0')
    return aa
import struct
def hex2float(fp32):
    y=int(fp32,16)
    z=struct.unpack('<f',struct.pack('<I',y))
    # print(z[0])
    return z[0]
import struct

def ieee754_to_float(binary_str):
    # 确保输入是32位的二进制字符串
    if len(binary_str) != 32:
        raise ValueError("输入的二进制字符串必须是32位的。")

    # 将二进制字符串转换为整数
    int_repr = int(binary_str, 2)

    # 将整数转换为四个字节
    bytes_repr = struct.pack('!I', int_repr)

    # 将四个字节解释为浮点数
    float_repr = struct.unpack('!f', bytes_repr)[0]

    return float_repr

def to_signed_32_bit(n):
    return (n & 0xFFFFFFFF) - (0x100000000 if n & 0x80000000 else 0)





In [2]:
def twos_complement(bin_str):
    # 如果输入为全0, 则直接返回原字符串
    if all(ch == '0' for ch in bin_str):
        return bin_str
    
    # 对字符串中的每个字符取反
    inverted_str = ''.join('1' if ch == '0' else '0' for ch in bin_str)
    #print(type(inverted_str))
    # 将取反后的字符串转为整数，并加1
    inverted_num = int(inverted_str, 2) + 1
    
    # 将结果转回二进制字符串
    return bin(inverted_num)[2:].zfill(len(bin_str))

In [3]:
                        #exp max模块
def expmax_moudle(exp):
    expMax = exp[0]
    for i in range(64):
        if(expMax < exp[i]):
                expMax = exp[i]
    return expMax

In [4]:
                        #shift orchest模块
def shiftOrchest_moudle(w_manti_list):
    sewo = [[""] * 64 for _ in range(24)]
    for i in range(23,-1,-1):
        for j in range(64):
            sewo[23-i][j] = w_manti_list[j][i]
            #print('sewo[',i,'][',j,'] = manti[',j,'][',i,']')
    '''for k in range(24):
        print(sewo[k])'''
    #print(sewo[0])

    #print(sewo[0])
    return sewo

In [5]:
                    #decoder模块
def decoder_moudle(sewo,a_manti,sign,i):
    shift_a_manti = [""] * 64
    int_shift_a_manti = [0]*64
    result_num = 0
    result_num_list = []
    result_str = [""] * 8
    #print('sewo[', i,'] = ',sewo)

    
    
    for k in range(64):     #把a_manti进行移位，变补码
        if sign[k] == 0:
            shift_a_manti[k] = a_manti[k][:i+1]
            shift_a_manti[k] = shift_a_manti[k].rjust(32,'0')
            int_shift_a_manti[k] = int(shift_a_manti[k],2)
        else:
            shift_a_manti[k] = a_manti[k][:i+1]
            shift_a_manti[k] = twos_complement(shift_a_manti[k])
            shift_a_manti[k] = shift_a_manti[k].rjust(32,'1')
            int_shift_a_manti[k] = int(shift_a_manti[k],2)
        ##print('shift_a_manti[',k,'] = ',bin(int_shift_a_manti[k])[2:])
    
    
    for b in range(8):      #第几个窗口
        result_num = 0
        for c in range(8):  #窗口里的哪个数
            #print(int(sewo[8 * b + c]) == 1)
            if int(sewo[8 * b + c]) == 1 :
                #print('int_shift_a_manti[',8 * b + c,' ] = ',bin(int_shift_a_manti[8 * b + c]))
                result_num = result_num + int_shift_a_manti[8 * b + c]
                #print('before result_num  ',8 * b + c,' = ',result_num)
                result_num = str(bin(result_num)[2:]).rjust(32,'0')
                result_num = result_num[-32:]
                result_num = int(result_num,2)
                #print('result_num [',8 * b + c,' ] = ',bin(result_num)[2:])
                ################################加完移除问题
            else:
                result_num = result_num + 0
        result_num_list.append(result_num)
    #print(result_num_list)


    return result_num_list

In [6]:
                        #adder tree模块
def adder_Tree_moudle(outnum):
    result = 0
    for a in range(8):
        for b in range(24):
            result = result + outnum[b][a]
    result = str(bin(result)[2:])
    result = result[-32:]
    result = int(result,2)
    #print('result = ',result)
    #print('result is ',twos_complement(str(bin(result)[2:])))
    return result

In [7]:
def post_process_moudle(result,emax):
    data_sign = str(bin(result)[2:])[0]
    #print('data_sign = ',data_sign)
    if(int(data_sign) == 1):
        result = twos_complement(str(bin(result)[2:]))
        result_str = '1' + result[3:]
        result = result_str
        # print('result = ',result)
        
    cnt = 0
    offste = 0
    data_exp = 0
    for a in range(32):
        if(int(result[a]) == 0):
            cnt = cnt +1
        else:
            break
    #print('cnt = ',cnt)
    if(cnt < 8):
        result = result[cnt + 1:cnt + 24]
        data_manti = result
        offste = 8 - cnt
        data_exp = str(bin(emax - 127 + offste)[2:]).ljust(8,'0')
        #print('offset = ',offste)
        print('data_manti = ',result)
        #print('data_exp = ',data_exp)
    else:
        result = result[cnt:].ljust(24,'0')
        offste = cnt - 8
        data_exp = emax - 127 - offste
    data = data_sign + str(data_exp) + data_manti
    return data


def post_process(result, expMax):
    sign = 0
    result = BitArray(int=result, length=32)
    if(result.int < 0):
        result.invert()
        sign = 1
    # result = BitArray(int=int(result), length=32)
    result = result.bin
    cnt = 7
    for i in range(1,32):
        if(result[i] == "0"):
            cnt = cnt - 1
        else:
            result = result[i+1:]
            break

    if(cnt>=0):
        result = result[0:23]
    else:
        result = result.ljust(23,'0')

    exp = BitArray(int=(expMax - 127 + cnt), length=9).bin
    fp32 = BitArray("0b"+str(sign)+exp[1:]+result).hex
    return fp32


In [8]:
                        #输入预处理
import numpy as np

# weight = np.array([
#         0.7529411912, 0.8196078539, 0.3019607961, 0.7960784435, 0.8666666746, 0.3411764801, 0.7803921700, 0.8431372643, 0.3215686381, 0.7686274648, 
#         0.8313725591, 0.3019607961, 0.8078431487, 0.8745098114, 0.3333333433, 0.8156862855, 0.8823529482, 0.3411764801, 0.8313725591, 0.8980392218, 
#         0.3490196168, 0.9019607902, 0.9686274528, 0.4196078479, 0.7764706016, 0.8313725591, 0.2941176593, 0.8941176534, 0.9490196109, 0.4196078479, 
#         0.8156862855, 0.8588235378, 0.3568627536, 0.8078431487, 0.8470588327, 0.3647058904, 0.8941176534, 0.9215686321, 0.4745098054, 0.8549019694, 
#         0.8823529482, 0.4431372583, 0.8431372643, 0.8549019694, 0.4352941215, 0.8470588327, 0.8588235378, 0.4392156899, 0.8705882430, 0.8823529482, 
#         0.4627451003, 0.7882353067, 0.8117647171, 0.3960784376, 0.8352941275, 0.9019607902, 0.3921568692, 0.8431372643, 0.9098039269, 0.3921568692, 
#         0.8000000119, 0.8627451062, 0.3450980484, 0.7921568751
#         ])

# active = np.array([
#        -0.0739838630, 0.0000000000, -0.0000000000, 0.0911507085, -0.0168761816, 0.0980103165, 0.0027172870, -0.7274691463, 0.0000000000, -0.0435802005, 
#         -0.0000000000, -0.0000000000, 0.0000000000, -0.1491651833, 0.0687774867, 0.0000000000, -0.4119710326, 0.0000000000, 0.0263345968, 0.0000000000, 
#         -0.0000000000, 0.0607688501, 0.0000000000, 0.0000000000, -0.0071072457, -0.4448229671, 0.0000000000, -0.1656140089, -0.0000000000, -0.0000000000, 
#         -0.0000000000, -0.0000000000, -0.0000000000, -0.0287525505, 0.0378144085, -0.0521513298, -0.1811170131, -0.1550923139, 0.1051603556, -0.0305253938, 
#         -0.0000000000, 0.0000000000, 0.0000000000, 0.0876532122, 0.0000000000, -0.2109717727, -0.1750611514, 0.0337304771, -0.0000000000, -0.2422664911, 
#         0.1072906852, -0.0000000000, 0.0125567913, 0.0201057401, 0.0000000000, 0.0000000000, -0.0000000000, -0.0000000000, -0.0000000000, -0.0000000000, 
#         -0.0897690207, -0.0000000000, -0.0000000000, 0.0000000000
#         ])

              # T1
# weight = np.array([

#        ])

# active = np.array([

#        ])


              # T1
# weight = np.array([
#       0.7277249903, 0.3364988920, -0.9805516258, 0.5200796318, 0.0482529689, 0.2728631004, 0.4908184433, 0.3175937269, -0.3598528843, -0.1134648934, 0.8724407107, 0.1765202993, 0.4745879724, 0.5334136908, 0.5466126463, 0.5314171171, -0.6117804260, -0.6235724215, 0.8227933689, 0.0086776752, 0.7609582254, -0.7839730790, 0.5473188322, 0.7879591442, -0.2423462451, -0.1341104243, 0.0251150912, -0.6239198816, 0.5096647305, 0.9787823314, -0.1808655050, 0.4532944923, 0.4744397689, 0.8527546927, 0.2468169656, -0.8536881809, -0.7030156160, -0.9684961186, -0.8568212847, -0.6050360532, -0.2185526454, 0.2705493933, 0.6535906374, 0.8854165350, -0.0134447102, 0.5734617399, 0.2658606039, -0.9873339067, -0.1409458981, -0.4875449473, -0.8540026931, 0.6768147058, 0.7691804211, 0.7151422834, 0.0376418765, 0.4071766895, 0.5399641477, -0.0004525387, 0.1607587600, 0.5967310651, 0.5039206032, -0.7586099763, 0.2655925386, -0.9274954738

#        ])

# active = np.array([
#        0.3396418993, -0.4318617665, -0.4319720684, -0.5723976126, -0.7519465227, -0.5804361372, 0.2440440861, -0.1963876883, 0.6640564988, 0.6825878784, -0.3907368263, -0.6906643930, -0.4504565286, -0.3244299262, -0.9731743386, -0.9411757524, 0.6153476514, 0.2577565321, 0.1047148914, 0.8919663465, 0.7774468564, -0.4248173206, 0.8889037913, 0.1892516346, -0.2609721474, -0.2821673754, 0.0639153191, -0.3166248404, -0.6641070150, 0.9860201942, -0.6111175913, 0.1290871624, 0.2880706310, -0.6002496067, -0.4429479007, 0.7218722364, 0.5390996304, -0.4910531420, -0.0800344502, -0.8303655232, 0.7458208742, 0.6814137158, -0.3910126608, -0.8237812707, 0.0921536320, -0.8452832795, -0.8964597733, -0.7893374443, 0.7239239737, 0.5059078446, -0.6076238484, -0.4936999672, 0.9202143196, -0.7389971299, -0.2404622829, 0.9755527931, 0.4606248371, 0.3554210469, -0.7263982204, -0.6482778221, 0.9236719956, -0.1947890169, 0.8995199233, 0.5984059856

#        ])

              # T2
weight = np.array([
-0.4001313152, -0.1169926101, 0.5271095954, -0.1671915165, -0.5165169101, 0.3314212591, 0.3610113727, 0.9741946572, -0.6880841449, -0.5563268077, 0.9093619391, 0.3033898582, -0.4781485196, -0.4657857171, 0.5944336046, 0.9816803513, -0.6963271328, -0.4637294138, 0.3793426064, 0.1034029204, 0.7503182455, -0.5266183965, -0.7299077872, -0.1941027248, -0.7137870395, 0.2760237632, -0.4911584314, 0.5347626018, 0.1750511866, 0.4240417518, 0.0237659199, -0.4310299919, -0.3237977467, 0.3946917220, 0.4775339703, 0.7647934151, -0.8850421715, -0.7666069417, -0.8068769097, 0.4604720175, 0.8049668675, 0.7846035302, 0.6339300846, 0.0725225533, -0.8329546552, -0.1076632133, -0.6886401447, 0.9791908006, 0.5343265938, 0.6630278447, 0.3978500716, -0.1478934692, -0.8540010410, -0.4108225692, -0.7654974528, -0.7355267059, 0.5428624286, 0.0323052765, -0.4812703468, -0.0646992620, -0.6922986445, 0.3002504833, -0.6672839615, 0.9331605711

       ])

active = np.array([
0.1032056727, 0.1509263983, -0.2704960732, 0.2466157978, 0.4268008535, -0.3500083049, 0.8894226837, 0.8799885641, 0.2806258119, -0.5847197035, -0.7876066634, 0.6694460128, -0.1739329052, 0.5800230575, 0.2367745398, -0.5749713491, -0.2055611781, 0.0093027928, -0.5505525294, 0.9234092250, -0.2003805478, 0.0427101575, -0.3866675252, -0.8065686007, -0.4785689208, -0.5739530753, -0.2217404104, 0.2690458215, 0.5426319519, 0.6856444077, -0.7947440168, -0.1961717876, -0.7176284025, 0.9518091535, -0.7559855072, -0.2886534700, 0.6067066515, 0.3100619598, 0.2583681372, -0.4646846648, 0.4294937084, -0.9249032883, -0.8898548817, -0.4869234042, 0.2751554843, 0.7548826159, -0.9805729819, 0.5274402776, -0.4490729166, 0.9967880065, -0.9057137618, 0.8298926497, 0.1404231718, 0.8095447043, 0.4374798752, 0.5288886058, 0.0340478196, 0.1123357339, -0.5251455335, -0.1313380461, 0.7121679950, -0.9394985755, -0.8515301523, -0.4152950443

       ])

              # T3
# weight = np.array([
# 0.6327372701, -0.7385217994, 0.5995132860, 0.2766261416, 0.2038718964, -0.1250540638, 0.0012103494, -0.1805168699, -0.1117911211, -0.0321272168, -0.6699735034, 0.1433150060, -0.2077733641, -0.7562825664, 0.8001927602, -0.1617515335, -0.1288774009, 0.7231065265, 0.3648667530, 0.1247916417, -0.4628012425, 0.3317616677, -0.7909828376, -0.8061587888, -0.8431814507, -0.2487215117, 0.5890484256, 0.3730707166, -0.0387256180, -0.5803395154, 0.4282125417, -0.8863915559, -0.1300440643, 0.7217904557, 0.0375322002, -0.5239926253, -0.6870503218, 0.3037345135, 0.7956160750, -0.7289502512, 0.9571141017, -0.1310877427, -0.1648734844, -0.3618397137, -0.7266098995, 0.5473433851, -0.3854095453, 0.4165819524, -0.7943433360, -0.5937319992, 0.1596053045, 0.9469351919, 0.5540625241, -0.2750431994, 0.9158607984, -0.3186585816, 0.1062032827, 0.4115030673, -0.2220118915, -0.8954258231, 0.3245278754, 0.8682656859, 0.0204050806, -0.4146371393

#        ])

# active = np.array([
# -0.2688754709, -0.2298294600, -0.8651077595, -0.8280683740, -0.5735684058, -0.4659463032, 0.4457678072, -0.2172169717, 0.0460447620, 0.5178971597, 0.3766098025, -0.9863240413, 0.4994788104, -0.0923189420, -0.9729698758, 0.9905746900, 0.7616937844, -0.6224218122, 0.3204712416, -0.1646493079, 0.3490853637, 0.3665477145, 0.8808539879, -0.5578823282, 0.7430605292, 0.2456365738, 0.7399478255, 0.7207103797, -0.7266479844, 0.5376142856, -0.8660560541, -0.5998219433, 0.9821868841, 0.6234925396, -0.6749343299, -0.9154638298, -0.6224207882, 0.6516591109, -0.6462336672, -0.4861858032, -0.2603547240, 0.2994763340, -0.9995045679, -0.1263096290, 0.2535372992, 0.3444769531, -0.2381241456, 0.8378640985, 0.4723676253, 0.1975406235, 0.0387553227, 0.7122484725, 0.2862384323, 0.7972146660, 0.0999659796, -0.9787221673, 0.0687286064, -0.1251006888, -0.6943689904, -0.6674286141, 0.1838361308, 0.4770809476, -0.3182285505, -0.1008771099

#        ])

              # T4
# weight = np.array([
# 0.2883787391, -0.7105056978, -0.3091364442, -0.6653444520, -0.7419085357, 0.9169302038, 0.6096382923, 0.3969429652, -0.8459155442, 0.4033004467, 0.1740157666, -0.5404730802, 0.0391438203, -0.1610790319, 0.4603129002, 0.5735577960, -0.4016600956, -0.4487451592, 0.6784947084, -0.9781340662, 0.1712096618, 0.5376378535, 0.3164255098, -0.3606679150, -0.0514050675, 0.5948748635, -0.3334277504, -0.3138059686, -0.3658677666, 0.8947502049, -0.0637772053, -0.8465956701, 0.4765007022, -0.0561261972, -0.6420215971, -0.1539823049, 0.3218970481, -0.3134370898, -0.1525376970, -0.7306537484, -0.6881612001, 0.5048162930, -0.9267091043, -0.9308011567, 0.1091268900, -0.0072031305, 0.7124984022, -0.1114258733, -0.3899217459, 0.4438556948, 0.0482545661, -0.8697446823, 0.7688583252, -0.4276378654, 0.6899661721, 0.9667574942, -0.5932437253, -0.8068217391, 0.7970752343, 0.4817494673, -0.6496051004, -0.7459567531, -0.2405639741, -0.1979145373

#        ])

# active = np.array([
# 0.3406724439, 0.6141569734, -0.7995132493, 0.6203441836, 0.0491928233, -0.9698914007, 0.4663703165, 0.1718573509, -0.4101680569, 0.3882360484, -0.6205733021, -0.1899707059, -0.9223310170, -0.0560891122, 0.1578095502, 0.6919857360, -0.0262270188, 0.5436990717, -0.1590304575, 0.8262950642, 0.8477262943, -0.7337553466, 0.4385528990, -0.4296879360, -0.7490975809, -0.0067157021, -0.0453957025, -0.7521285182, 0.8873953013, -0.4417201877, 0.4927661870, 0.1736140441, 0.3906484744, -0.5774104304, 0.4632632440, -0.1428404583, 0.2279504618, 0.0831910312, -0.9109512713, -0.4454511462, -0.0198913627, -0.3682360356, 0.2780321397, 0.6931002374, -0.4420882952, 0.0293555866, -0.7790961444, -0.9435566623, 0.5939124350, -0.7427483857, 0.2925750717, 0.6139591104, 0.7504526040, 0.0274073614, 0.0013261125, 0.0732539819, 0.8702026611, 0.5354454768, -0.6982666687, 0.8349942879, 0.4063426219, 0.4559524260, 0.2253928525, -0.1225865423

#        ])

              # T5
# weight = np.array([
# 0.4418154049, -0.9657322577, 0.1792325822, 0.1222342210, 0.7793273489, -0.7942921624, 0.0166375366, -0.4835073148, 0.9021325086, 0.3212833491, -0.6159139782, 0.6665323650, 0.3732683432, 0.0024857874, 0.5283813920, 0.6458343558, -0.4874009176, 0.7446905308, -0.8695888065, -0.9944302778, 0.0522905203, 0.0314953288, -0.1050599970, 0.5908171036, 0.3502509299, -0.0777823101, -0.7511511735, -0.0411603006, 0.2458966138, -0.2110100124, -0.0658072779, -0.6553025251, -0.4807878968, -0.0453315966, 0.5688284501, -0.2583214430, -0.5208873027, 0.3545937665, -0.8028659654, 0.0376284581, -0.9241503540, -0.2451777088, -0.7178394677, -0.1818469995, -0.3762174473, 0.2673539639, 0.5238034930, -0.4876689655, -0.9237410040, -0.0455211291, -0.0918634998, 0.8948706393, -0.5330935210, -0.9593878152, -0.2708155639, -0.0462533910, -0.2032329956, 0.9711259216, 0.6939863022, 0.6626735002, -0.9673510878, -0.9685107974, 0.2942619161, 0.3327634950

#        ])

# active = np.array([
# -0.5364595679, 0.9597094320, 0.5812537319, -0.2102647805, 0.4364521948, 0.3070950925, 0.0755501764, -0.5698460546, -0.5522547563, 0.7150837181, -0.2311343583, 0.5540136726, -0.6487888799, 0.1261783928, -0.7849610205, -0.9715754437, -0.8095560624, -0.9343175426, 0.1290092943, -0.2063115785, 0.9760603890, 0.4402037950, -0.3749214511, 0.4166709162, 0.5176634979, -0.1027873534, 0.6694037249, -0.9390569389, -0.7980204666, -0.0373511435, 0.2968165286, 0.0495855429, -0.0794638783, 0.6231825331, -0.1328153717, -0.0141611747, -0.5481648897, -0.1419154479, -0.4278641694, 0.1596052273, 0.4621152355, 0.2311216936, -0.4682815717, 0.0126012974, 0.8843218346, -0.3523037617, -0.2937988167, -0.4067466258, 0.7831298659, 0.1091412604, 0.5301268732, -0.6027788019, 0.3108337993, 0.5797687764, -0.6746087479, 0.5385065482, -0.8626951268, -0.9566872524, 0.1274403815, 0.0490471284, -0.8219424736, -0.3349560827, 0.5552262806, -0.4977287442

#        ])

              # T6
# weight = np.array([
# -0.7635964659, 0.0918889436, 0.5193786779, -0.0235927173, -0.3028776155, 0.9224079200, -0.8147200564, -0.6389444490, 0.8251545752, 0.5355935811, -0.1516913808, 0.3849788869, 0.1109024887, -0.8560841875, 0.1392787456, -0.1604736686, -0.6527465825, -0.5256755586, -0.6450744923, -0.9672650244, 0.9611648567, -0.1901017607, -0.4650632921, -0.0188045189, 0.1354513226, 0.0129440384, 0.3661939455, 0.2882652222, -0.9213320749, 0.1284604673, 0.6000893568, -0.0553136455, 0.6328337110, -0.9689945469, -0.2135040683, -0.9525178002, 0.6888820282, -0.0713181835, -0.9868515203, 0.1597246925, -0.4297920809, -0.6651864703, 0.7117650124, 0.1349360247, -0.0198223709, 0.1985800214, -0.8984414325, 0.1143076394, -0.8582665643, -0.8775415392, -0.9920060001, 0.9960938580, 0.3093088930, -0.4557860496, -0.6904995108, -0.6132124890, 0.5791848115, -0.3751306380, -0.5687481389, 0.8221806011, -0.5811382008, -0.5125033951, -0.9760274425, 0.2965028291

#        ])

# active = np.array([
# 0.8157527414, 0.0694081627, -0.3285935441, 0.8934346839, 0.8379869252, 0.6958153068, -0.8739489133, -0.2953939478, 0.3001347555, -0.1140663605, 0.7143063898, -0.6336003651, -0.2657464225, 0.8143940881, 0.9818738611, 0.4984569010, -0.7776660407, 0.2817632265, 0.8924366334, 0.8631172537, -0.5646600321, -0.8152491931, 0.0724477849, 0.3572318157, -0.0423810696, -0.5916397910, 0.3870207182, -0.0787222857, -0.3608350065, -0.3748723726, -0.3960830708, 0.0412602683, -0.1447827676, 0.4267071444, -0.7820144993, 0.8115010065, -0.9903172288, -0.6558470070, -0.2117818542, -0.7570296354, -0.1446258894, 0.3610979976, -0.0730981909, 0.5802706743, 0.0766516413, 0.1885398292, -0.4751611710, 0.6062664158, 0.4217197660, 0.1800788918, 0.7777385638, -0.0397471863, -0.4661894898, -0.2011745771, -0.9205964680, -0.0906440052, -0.7823942857, 0.7771949419, 0.9924432202, 0.6637163689, 0.8804932668, -0.1677934283, -0.4355198737, 0.1742368912

#        ])

              # T7
# weight = np.array([
# -0.3989438948, -0.0983410115, -0.4315669950, -0.0313994432, 0.2149267278, 0.9863648713, -0.1098956402, -0.0515240249, -0.3059122155, -0.1914547007, 0.4807684789, 0.0311692997, -0.1027142582, 0.5595581497, -0.9554009474, 0.0539963231, 0.9468747381, -0.8003034835, 0.7746825709, -0.6569851156, -0.3369314961, 0.2699730623, 0.6611814693, -0.7260271465, -0.8042077995, -0.1305141674, -0.4111020619, -0.6345191391, 0.7318817226, -0.3345981960, 0.9270565929, -0.6485943620, 0.1667492229, -0.5188411211, 0.3380097846, 0.5615405555, -0.3424894494, -0.4455969754, 0.1050155065, -0.8214237658, -0.8200479476, -0.9728514763, 0.3396717546, -0.7691131723, 0.2923776563, -0.1247194151, 0.6507606730, 0.9363199435, 0.3133887109, 0.9560618594, 0.0718126554, -0.9887051295, -0.8857424520, -0.0648708869, -0.9861750697, 0.8114228289, -0.8870063317, 0.4283703250, 0.9342264426, 0.3508274239, -0.3573339900, 0.1435415039, -0.6151895689, -0.0742498981

#        ])

# active = np.array([
# -0.6459410603, 0.1276026508, 0.0441346602, -0.1539598432, 0.7938531874, 0.1290168240, 0.4061279010, -0.0327682378, 0.7416378464, -0.6373473127, -0.6478721111, -0.0955289407, 0.0409343448, -0.2738615698, -0.5612275088, -0.2853192972, -0.5463738326, -0.8201629408, -0.0336165079, 0.1818236534, 0.8445877420, -0.2633170503, 0.8834911323, -0.0742610174, 0.6427742798, -0.7993378318, 0.5131995614, 0.1257647113, 0.3951358312, 0.8046050712, -0.4276875451, -0.1088334691, 0.3471767433, 0.7263566067, 0.7298016381, 0.3009023766, 0.1627862882, 0.4511154644, 0.8694793144, -0.8135882581, -0.7903509304, -0.7290006682, -0.0002062239, -0.2927673895, 0.5853354003, -0.9839923258, -0.1079755632, 0.4255476488, 0.0507626593, -0.4145001498, -0.2612472050, -0.1480224572, 0.6042206373, -0.6252558709, -0.5768724572, -0.2020476694, 0.3868862562, -0.8561718335, -0.6618299336, 0.9853419281, 0.6544556139, -0.8598010706, 0.4310082291, -0.9993928363

#        ])

              # T8
# weight = np.array([
# -0.6712068529, 0.1941660810, -0.0953082205, -0.4510769438, 0.1042526189, 0.2780096153, -0.2182417767, 0.3204062931, 0.4541320154, -0.1136548749, -0.3405148086, -0.6074716676, 0.5868376663, -0.4140184581, -0.9212603539, 0.5344514769, 0.9704823177, -0.4273063784, 0.3068318916, -0.8092013675, -0.8157293228, -0.3098596211, 0.2742559661, 0.8846595644, -0.5741795671, 0.2995305516, -0.5762734242, 0.2574132134, 0.1945601178, -0.7209015875, 0.5140255266, 0.4744057054, 0.7214109686, 0.1407577486, 0.7470545432, -0.5242167062, 0.2012528978, -0.1246658195, -0.6485218955, 0.1411227583, -0.5234127609, -0.7456153338, 0.1377687584, 0.2062738850, 0.2174574054, 0.6673982305, 0.2892692702, -0.2945546472, 0.0928157483, 0.0263713894, 0.0888748743, 0.3304674494, 0.8993523164, -0.4652945937, 0.2104544540, -0.5748573166, -0.3465248144, -0.8882049288, 0.2695378650, 0.3738125089, -0.2120751258, -0.1490398330, -0.9676265214, 0.8437439133

#        ])

# active = np.array([
# 0.9453553850, -0.8455363240, -0.7914952927, -0.4360416325, -0.7980829377, -0.8180957012, -0.1263454720, 0.2697251354, 0.6037146880, -0.0706351040, -0.2018201358, 0.5564311384, 0.3085468406, -0.7883138761, 0.4943997109, -0.8827952737, -0.5373132280, -0.2997596833, 0.1577857187, -0.6069923656, -0.6580539744, 0.8899943366, 0.2838948646, 0.6657738035, 0.4503795175, 0.2377834876, -0.1647018735, 0.2339762949, -0.8489541033, 0.8092143130, 0.1960100220, -0.8277296401, -0.1313177996, -0.4320829366, -0.0729929508, -0.4638126988, -0.5765621279, 0.5969213596, 0.1794096047, 0.1499260209, -0.3720577564, 0.6980341400, 0.0404621975, -0.0023036481, -0.2662818809, 0.3959332177, -0.0344712129, 0.2421113832, 0.7278015555, 0.1108817402, -0.2908773140, 0.7678226793, -0.5093910327, 0.0170160563, 0.3526850687, 0.3782111379, -0.4053652497, 0.6951206016, -0.0375074239, 0.3445228631, -0.3137611651, -0.1333736547, -0.9642975898, 0.6727694040

#        ])

              # T9
# weight = np.array([
# -0.8501750769, 0.3152068509, 0.0904576436, -0.4852933484, -0.6348361371, 0.5605833411, -0.8483405370, -0.5958452261, 0.9665370351, 0.3634981043, -0.0553372783, -0.5189593310, 0.1845083792, 0.0299539446, 0.9180853308, 0.2183667150, 0.2825792748, -0.8173939030, 0.5049394567, 0.8073618184, 0.7214905044, 0.0542131827, -0.9069019744, 0.7395036996, 0.9719868591, 0.7299423534, 0.8878040419, -0.2159919374, 0.7797906165, -0.8999520154, 0.6430773996, -0.2954866639, 0.3885196209, 0.9011561374, 0.6736464704, -0.5502659755, -0.1581476977, 0.6657708230, 0.5024257976, -0.6087784182, -0.6984111917, -0.3146825765, -0.6369514563, -0.0220060415, 0.3785905989, -0.4633282178, -0.2917291708, 0.8942225668, -0.5173452012, 0.4433727608, -0.9004828321, 0.5338582333, -0.2671524989, 0.9618078198, -0.7801739747, -0.1845405840, -0.6493616286, -0.8793458141, -0.7060489582, 0.2006252107, -0.7712575173, -0.8648950256, -0.7658109954, 0.6550646000

#        ])

# active = np.array([
# 0.5611013115, 0.0645104260, -0.9766896052, -0.0514585878, -0.2894009880, -0.5271541856, 0.5521181603, -0.4856815857, 0.0910637937, 0.8523465702, -0.2095107025, 0.7814616718, -0.5477279786, -0.9291982614, 0.5955803501, -0.3972293950, 0.9865940378, -0.1708404332, 0.0027864972, -0.3907619286, -0.3251166635, 0.2314434271, -0.7481153093, 0.9136609582, 0.0960096441, 0.4171249314, -0.9834776816, 0.0272868436, 0.4087577965, 0.9311962249, 0.9825441929, -0.2964828672, 0.0745379019, 0.3100586944, -0.2880721043, 0.1829956807, -0.7733750495, 0.1942224404, 0.6021737002, 0.9075600742, 0.1606409275, 0.6560560825, -0.5345428588, -0.3810097296, 0.1179503051, 0.4415249004, -0.8114341146, 0.8021326999, 0.4419676321, -0.8321799580, 0.4934522441, 0.0933438523, -0.5878061200, 0.6510065681, -0.7186125896, 0.2949449347, -0.7299436375, -0.3258991583, -0.1922557236, 0.0122330600, -0.8942868666, 0.6577724861, 0.7189927794, 0.0018600999

#        ])

              # T10
# weight = np.array([
# 0.7277249903, 0.3364988920, -0.9805516258, 0.5200796318, 0.0482529689, 0.2728631004, 0.4908184433, 0.3175937269, -0.3598528843, -0.1134648934, 0.8724407107, 0.1765202993, 0.4745879724, 0.5334136908, 0.5466126463, 0.5314171171, -0.6117804260, -0.6235724215, 0.8227933689, 0.0086776752, 0.7609582254, -0.7839730790, 0.5473188322, 0.7879591442, -0.2423462451, -0.1341104243, 0.0251150912, -0.6239198816, 0.5096647305, 0.9787823314, -0.1808655050, 0.4532944923, 0.4744397689, 0.8527546927, 0.2468169656, -0.8536881809, -0.7030156160, -0.9684961186, -0.8568212847, -0.6050360532, -0.2185526454, 0.2705493933, 0.6535906374, 0.8854165350, -0.0134447102, 0.5734617399, 0.2658606039, -0.9873339067, -0.1409458981, -0.4875449473, -0.8540026931, 0.6768147058, 0.7691804211, 0.7151422834, 0.0376418765, 0.4071766895, 0.5399641477, -0.0004525387, 0.1607587600, 0.5967310651, 0.5039206032, -0.7586099763, 0.2655925386, -0.9274954738

#        ])

# active = np.array([
# 0.3396418993, -0.4318617665, -0.4319720684, -0.5723976126, -0.7519465227, -0.5804361372, 0.2440440861, -0.1963876883, 0.6640564988, 0.6825878784, -0.3907368263, -0.6906643930, -0.4504565286, -0.3244299262, -0.9731743386, -0.9411757524, 0.6153476514, 0.2577565321, 0.1047148914, 0.8919663465, 0.7774468564, -0.4248173206, 0.8889037913, 0.1892516346, -0.2609721474, -0.2821673754, 0.0639153191, -0.3166248404, -0.6641070150, 0.9860201942, -0.6111175913, 0.1290871624, 0.2880706310, -0.6002496067, -0.4429479007, 0.7218722364, 0.5390996304, -0.4910531420, -0.0800344502, -0.8303655232, 0.7458208742, 0.6814137158, -0.3910126608, -0.8237812707, 0.0921536320, -0.8452832795, -0.8964597733, -0.7893374443, 0.7239239737, 0.5059078446, -0.6076238484, -0.4936999672, 0.9202143196, -0.7389971299, -0.2404622829, 0.9755527931, 0.4606248371, 0.3554210469, -0.7263982204, -0.6482778221, 0.9236719956, -0.1947890169, 0.8995199233, 0.5984059856

#        ])








#print('weight = ',weight[0])
#print('active = ',active)

                        #软件模拟结果   
mac_result_software = 0
for a in range(64):
    #print(a)
    multiply_mid = weight[a] * active[a]
    mac_result_software = multiply_mid + mac_result_software
print('the mac result is ',mac_result_software)

                        #预处理           

exp = [0] *64
weight_ieee754 = [""] * 64
active_ieee754 = [""] * 64

                        #处理exp 符号
sign = [""] * 64
for b in range(64):     
        weight_ieee754[b] = str(ieee754B(weight[b])) 
        active_ieee754[b] = str(ieee754B(active[b]))
        e_w = weight_ieee754[b][1:9]
        e_a = active_ieee754[b][1:9]
        exp[b] = int(e_w, 2) + int(e_a, 2)
        sign[b] = int(weight_ieee754[b][0]) ^ int(active_ieee754[b][0])
#print(weight_ieee754[0])
                        #选出emax
'''for z in range(64):
       print('sign',z,'is ',sign[z])'''
expMax = expmax_moudle(exp)     
#print('expmax is ',expMax)
#print(type(expMax))


w_manti_list = []
a_manti_list = []
for c in range(64):
        
        w_manti = "1" + weight_ieee754[c][9:]
        w_manti = w_manti[:24-(int(expMax-exp[c]))]
        w_manti = w_manti.rjust(24,'0')
        w_manti_list.append(w_manti)
        

        a_manti = "1" + active_ieee754[c][9:]
        a_manti_list.append(a_manti)
        
# for z in range(64):
#        print('w_manti_list[',z,'] = ',w_manti_list[z])

sewo = shiftOrchest_moudle(w_manti_list)
#for z in range(24):
#print(len(w_manti_list[0]))
'''for z in range(24):
       print(w_manti_list[z])'''
outnum = []
for d in range(24):
       outnum.append(decoder_moudle(sewo[d],a_manti_list,sign,d))
       # print(outnum)

# for z in range(24):
#        print(bin(outnum[z])[2:])
result = adder_Tree_moudle(outnum)
result = to_signed_32_bit(result)

data = post_process(result,expMax)
data = hex2float(data)
#print('binnary_data = ',data)

print('the softresult result is ',data)
print(data - mac_result_software)
# print('datatype = ',type(data))
# print('datalen = ',len(data))
print('end')
        

the mac result is  -2.0322600414928385
the softresult result is  -2.0322580337524414
2.0077403970475416e-06
end
